In [48]:
import pandas as pd

df_rcc = pd.read_excel('MLDataset.xlsx', sheet_name='result_constituencies_Candidate') 
df_rcp = pd.read_excel('MLDataset.xlsx', sheet_name='result_constituencies_PartyList') 
df_rcs = pd.read_excel('MLDataset.xlsx', sheet_name='result_constituencies_status') 
df_ipo = pd.read_excel('MLDataset.xlsx', sheet_name='info_party_overview') 

In [34]:
df_con = pd.concat([df_rcc,df_rcp], axis=1) #Merge df_rcc with df_rcp
df_con

,cons_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,cons_id,party_id,party_list_vote,party_list_vote_percent
0,BKK_1,2.0,18261.0,19.34285,763.0,BKK_1,701,3573,3.78468
1,BKK_1,4.0,11544.0,12.22791,701.0,BKK_1,702,11,0.01165
2,BKK_1,6.0,1495.0,1.58357,743.0,BKK_1,703,10,0.01059
3,BKK_1,14.0,130.0,0.13770,733.0,BKK_1,705,13857,14.67794
4,BKK_1,9.0,625.0,0.66203,719.0,BKK_1,706,1675,1.77423
...,...,...,...,...,...,...,...,...,...
2206,NaN,NaN,NaN,NaN,NaN,BKK_9,780,44,0.04082
2207,NaN,NaN,NaN,NaN,NaN,BKK_9,781,46,0.04267
2208,NaN,NaN,NaN,NaN,NaN,BKK_9,783,2,0.00186
2209,NaN,NaN,NaN,NaN,NaN,BKK_9,784,74,0.06865


In [49]:
df_rcc_rc = df_rcc
df_rcp_rc = df_rcp
df_rcc_rc.drop(columns=['cons_id','mp_app_vote_percent','mp_app_rank'],inplace =True)
df_rcp_rc.drop(columns=['cons_id','party_list_vote_percent'],inplace=True)
df_rcc_rc

,mp_app_vote,party_id
0,18261,763
1,11544,701
2,1495,743
3,130,733
4,625,719
...,...,...
493,2372,701
494,466,769
495,18866,763
496,2475,743


In [50]:
import seaborn as sns       
sns.pairplot(df_cons)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
import numpy as np

count_cons_rcc = df_rcc['cons_id'].value_counts()
count_cons_rcp = df_rcp['cons_id'].value_counts()
max_count = max(count_cons_rcc.max(),count_cons_rcp.max())
for category in count_cons_rcc.index:
    if count_cons_rcc.get(category, 0) < max_count:
        additional_rows = max_count - count_cons_rcc.get(category, 0)
        df_rcc = pd.concat([df_rcc, pd.DataFrame({'cons_id': [category] * additional_rows, 'mp_app_vote': [None] * additional_rows , 'mp_app_vote_percent': [None] * additional_rows, 'party_id': [None] * additional_rows})], ignore_index=True)
df_rcc['cons_id_number'] = df_rcc['cons_id'].str.extract(r'BKK_(\d+)').astype(int)
df_rcc = df_rcc.sort_values(by='cons_id_number').drop(columns='cons_id_number')

party_id_mapping = df_rcp.groupby('cons_id')['party_id'].apply(list).to_dict()

# Function to fill missing 'party_id' values in df1 based on the mapping
def fill_missing_party_id(row):
    if pd.isna(row['party_id']):
        cons_id = row['cons_id']
        if cons_id in party_id_mapping:
            available_party_ids = party_id_mapping[cons_id]
            if available_party_ids:
                return available_party_ids.pop(0)
    return row['party_id']

# Apply the fill_missing_party_id function to fill missing 'party_id' values in df1
df_rcc['party_id'] = df_rcc.apply(fill_missing_party_id, axis=1)

# df_rcc
df_rcc.to_excel('df_rcc_real.xlsx',index=False)

In [47]:
df_rcc_sorted = df_rcc.sort_values(by=['cons_id', 'party_id'], ascending=True)
df_rcc_sorted.to_excel('df_rcc_real1.xlsx',index=False)
# df_con_reindex = pd.concat([df_rcc_sorted,df_rcp], axis=1)
# df_con_reindex